In [4]:
from tkinter import *
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import filedialog
import cv2
import numpy as np
def add_label(name):
    _label = tk.Label(window,bg='#10cfc9' ,text = name, font=('Arial', 20))
    _label.pack()
    return _label
window = tk.Tk()
window.geometry("500x300")
window.configure(background='#10cfc9')
header_label = add_label('特效修圖大師')

def ripple_effect( f, method, amplitude, period ):
	nr, nc = f.shape[:2]
	map_x = np.zeros( [nr, nc], dtype = 'float32' )
	map_y = np.zeros( [nr, nc], dtype = 'float32' )
	x0, y0 = nr // 2, nc // 2
	for x in range( nr ):
		for y in range( nc ):
			if method == 1:		# x-direction
				xx = np.clip( x + amplitude * np.sin( x / period ), 0, nr - 1 )
				map_x[x,y] = y
				map_y[x,y] = xx
			elif method == 2:	# y-direction
				yy = np.clip( y + amplitude * np.sin( y / period ), 0, nc - 1 )
				map_x[x,y] = yy
				map_y[x,y] = x
			elif method == 3:	# x & y direction
				xx = np.clip( x + amplitude * np.sin( x / period ), 0, nr - 1 )
				yy = np.clip( y + amplitude * np.sin( y / period ), 0, nc - 1 )
				map_x[x,y] = yy
				map_y[x,y] = xx
			else:				# Radial
				r = np.sqrt( ( x - x0 ) ** 2 + ( y - y0 ) ** 2 )
				if r == 0:  theta = 0
				else:		theta = np.arccos( ( x - x0 ) / r )
				r = r + amplitude * np.sin( r / period )
				if y - y0 < 0:  theta = -theta
				map_x[x,y] = np.clip( y0 + r * np.sin( theta ), 0, nc - 1 )
				map_y[x,y] = np.clip( x0 + r * np.cos( theta ), 0, nr - 1 )
	g = cv2.remap( f, map_x, map_y, cv2.INTER_LINEAR )
	return g
def fisheye_effect( f ):
	nr, nc = f.shape[:2]
	map_x = np.zeros( [nr, nc], dtype = 'float32' )
	map_y = np.zeros( [nr, nc], dtype = 'float32' )
	x0, y0 = nr // 2, nc // 2
	R = np.sqrt( nr ** 2 + nc ** 2 ) / 2
	for x in range( nr ):
		for y in range( nc ):
			r = np.sqrt( ( x - x0 ) ** 2 + ( y - y0 ) ** 2 )
			if r == 0:  theta = 0
			else:		theta = np.arccos( ( x - x0 ) / r )
			r = ( r * r ) / R
			if y - y0 < 0:  theta = -theta
			map_x[x,y] = np.clip( y0 + r * np.sin( theta ), 0, nc - 1 )
			map_y[x,y] = np.clip( x0 + r * np.cos( theta ), 0, nr - 1 )
	g = cv2.remap( f, map_x, map_y, cv2.INTER_CUBIC )
	return g
def RGB_to_HSI( R, G, B ):
	r = R / 255
	g = G / 255
	b = B / 255
	if R == G and G == B:  
		H = -1.0  
		S =  0.0
		I = ( r + g + b ) / 3
	else:                  
		x = ( 0.5 * ( ( r - g ) + ( r - b ) ) ) / \
		    np.sqrt( ( r - g ) ** 2 + ( r - b ) * ( g - b ) )
		if x < -1.0:  x = -1.0
		if x >  1.0:  x =  1.0
		theta = np.arccos( x ) * 180 / np.pi
		if B <= G:  
			H = theta
		else:
			H = 360.0 - theta
		S = 1.0 - 3.0 / ( r + g + b ) * min( r, g, b )
		I = ( r + g + b ) / 3
	return H, S, I

def HSI_to_RGB( H, S, I ):
	if H == -1.0:
		r = I
		g = I
		b = I
	elif H >= 0 and H < 120:
		HH = H
		b = I * ( 1 - S )
		r = I * ( 1 + ( S * np.cos( HH * np.pi / 180 ) ) /
			np.cos( ( 60 - HH ) * np.pi / 180 ) ) 
		g = 3.0 * I - ( r + b )
	elif H >= 120 and H < 240:
		HH = H - 120.0
		r = I * ( 1 - S )
		g = I * ( 1 + ( S * np.cos( HH * np.pi / 180 ) ) /
			np.cos( ( 60 - HH ) * np.pi / 180 ) )
		b = 3 * I - ( r + g )
	else:
		HH = H - 240
		g = I * ( 1 - S )
		b = I * ( 1 + ( S * np.cos( HH * np.pi / 180 ) ) /
			np.cos( ( 60 - HH ) * np.pi / 180 ) )
		r = 3 * I - ( g + b )
	rr = round( r * 255 )
	gg = round( g * 255 )
	bb = round( b * 255 )
	R = np.uint8( np.clip( rr, 0, 255 ) )
	G = np.uint8( np.clip( gg, 0, 255 ) )
	B = np.uint8( np.clip( bb, 0, 255 ) )
	return R, G, B

def HSI_processing( f, angle = 0, saturation = 100, intensity = 100 ):
	g = f.copy( )
	nr, nc = f.shape[:2]

	for x in range( nr ):
		for y in range( nc ):
			H, S, I = RGB_to_HSI( f[x,y,2], f[x,y,1], f[x,y,0] )
			H = H + angle
			if H > 360:  H = H - 360	
			S = S * saturation / 100
			I = I * intensity / 100
			R, G, B = HSI_to_RGB( H, S, I )
			g[x,y,0] = B
			g[x,y,1] = G
			g[x,y,2] = R
	return g    
def fisheyeimg():
    #initialdir 對話框開啟的目錄, title對話框的標題, filetypes找尋的副檔名
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    fromCenter = False
    showCrosshair = False
    r = cv2.selectROIs("Select Rois",img,fromCenter,showCrosshair)
    
    if len(r) >0:
        for ROI in r:
            x,y,w,h=ROI
        
            imCrop = img[int(y):int(y + h), int(x):int(x + w)]            
            imCrop2 = fisheye_effect( imCrop )
            cv2.imshow("Image", imCrop2)
            cv2.waitKey()
            cv2.destroyAllWindows()
            img2 = overlay_transparent(img, imCrop2,int(x),int(y))
            cv2.imshow("Image2", img2)
            cv2.imwrite( "fisheye_img.png", img2 )
            cv2.waitKey()
            cv2.destroyAllWindows()
def detailEnhanceimg():
    #initialdir 對話框開啟的目錄, title對話框的標題, filetypes找尋的副檔名
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    fromCenter = False
    showCrosshair = False
    r = cv2.selectROIs("Select Rois",img,fromCenter,showCrosshair)
    
    if len(r) >0:
        for ROI in r:
            x,y,w,h=ROI
        
            imCrop = img[int(y):int(y + h), int(x):int(x + w)]            
            imCrop2 = cv2.detailEnhance( imCrop )
            cv2.imshow("Image", imCrop2)
            cv2.waitKey()
            cv2.destroyAllWindows()
            img2 = overlay_transparent(img, imCrop2,int(x),int(y))
            cv2.imshow("Image2", img2)
            cv2.imwrite( "detailEnhance_img.png", img2 )
            cv2.waitKey()
            cv2.destroyAllWindows()
def pencilSketchimg():
    #initialdir 對話框開啟的目錄, title對話框的標題, filetypes找尋的副檔名
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    fromCenter = False
    showCrosshair = False
    r = cv2.selectROIs("Select Rois",img,fromCenter,showCrosshair)
    
    if len(r) >0:
        for ROI in r:
            x,y,w,h=ROI
        
            imCrop = img[int(y):int(y + h), int(x):int(x + w)]   
            gray = cv2.cvtColor(imCrop,cv2.COLOR_BGR2GRAY)
            img_blur = cv2.GaussianBlur(gray, ksize=(21, 21),sigmaX=0, sigmaY=0)
            imCrop2 = cv2.divide(gray, img_blur, scale=255)
            cv2.imshow("Go", imCrop2)
            
            cv2.imwrite( "pencilSketch_img.png", imCrop2 )
            cv2.waitKey()
            cv2.destroyAllWindows()
def rippleimg():
    #initialdir 對話框開啟的目錄, title對話框的標題, filetypes找尋的副檔名
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    fromCenter = False
    showCrosshair = False
    r = cv2.selectROIs("Select Rois",img,fromCenter,showCrosshair)
    
    if len(r) >0:
        for ROI in r:
            x,y,w,h=ROI
        
            imCrop = img[int(y):int(y + h), int(x):int(x + w)]            
            imCrop2 = ripple_effect( imCrop, 1, 5, 2 )
            cv2.imshow("Image", imCrop2)
            cv2.waitKey()
            cv2.destroyAllWindows()
            img2 = overlay_transparent(img, imCrop2,int(x),int(y))
            cv2.imshow("Image2", img2)
            cv2.imwrite( "rippleimg_img.png", img2 )
            cv2.waitKey()
            cv2.destroyAllWindows()   
def stylizationimg():
    #initialdir 對話框開啟的目錄, title對話框的標題, filetypes找尋的副檔名
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    fromCenter = False
    showCrosshair = False
    r = cv2.selectROIs("Select Rois",img,fromCenter,showCrosshair)
    
    if len(r) >0:
        for ROI in r:
            x,y,w,h=ROI
        
            imCrop = img[int(y):int(y + h), int(x):int(x + w)]            
            imCrop2 = cv2.stylization( imCrop, 1, 5, 2 )
            cv2.imshow("Image", imCrop2)
            cv2.waitKey()
            cv2.destroyAllWindows()
            img2 = overlay_transparent(img, imCrop2,int(x),int(y))
            cv2.imshow("Image2", img2)
            cv2.imwrite( "stylization_img.png", img2 )
            cv2.waitKey()
            cv2.destroyAllWindows() 
def light():
    #initialdir 對話框開啟的目錄, title對話框的標題, filetypes找尋的副檔名
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    img1 = HSI_processing( img, 0, 100, 150 )
    cv2.imshow( "HSI", img1 )
    cv2.imwrite( "HSIimage.png", img1 )
    cv2.waitKey()
    cv2.destroyAllWindows()
def overlay_transparent(background, overlay, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background
def combined( ):
    
    img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    img = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
    fromCenter = False
    showCrosshair = False
    r = cv2.selectROIs("Select Rois",img,fromCenter,showCrosshair)
    
    if len(r) >0:
        for ROI in r:
            x,y,w,h=ROI
        
            overlay_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))        
            imCrop2 = cv2.resize(cv2.imread(overlay_path), (70, 50),interpolation=cv2.INTER_AREA)
            cv2.imshow("Image", imCrop2)
            cv2.waitKey()
            cv2.destroyAllWindows()
            img2 = overlay_transparent(img, imCrop2,int(x),int(y))
            cv2.imshow("combined.png", img2)
            cv2.imwrite( "combinedimg.png", img2 )
            cv2.waitKey()
            cv2.destroyAllWindows()
def Matting( ):
    image_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
    image = Image.open(image_path)
    image = image.convert('RGBA')
    print(image.mode)

    newImage = []
    for item in image.getdata():
        if item[:3] == (255, 255, 255):
            newImage.append((255, 255, 255, 0))
        else:
            newImage.append(item)

    image.putdata(newImage)

    
    image.save('output.png')
    print(image.mode, image.size)
    
def usetalk():
    newWindow = tk.Toplevel(window)
    newWindow.geometry("500x300")
    text = tk.Text(newWindow, width=200, height=100)
    text.pack()
    text.insert("insert", "你需要點擊space或enter鍵，然後才能框選下一個目標。\n當你框選所有的目標後，你需要esc鍵\n以下是輸出圖片\n去背的圖片輸出是叫output.png，\n變亮是HSIimage.png\n疊放是combinedimg.png\n魚眼fisheye_img.png\n細節增強detailEnhance_img.png\n素描pencilSketch_img.png\n自由風格stylization_img\n漣漪ripple_img.png\n")




#建立讀取讀片按鈕 
button = tk.Button(window, text = "魚眼", width = 10, height = 2, command = fisheyeimg,bg='#c8c8c8',activeforeground='red')
button.place(x = 20, y = 50)

button1 = tk.Button(window, text = "細節增強", width = 10, height = 2, command = detailEnhanceimg,bg='#c8c8c8')
button1.place(x = 20, y = 100)

button2 = tk.Button(window, text = "素描", width = 10, height = 2, command = pencilSketchimg,bg='#c8c8c8')
button2.place(x = 20, y = 150)

button3 = tk.Button(window, text = "自由風格", width = 10, height = 2, command = stylizationimg,bg='#c8c8c8')
button3.place(x = 20, y = 200)

button4 = tk.Button(window, text = "去背", width = 10, height = 2, command = Matting,bg='#c8c8c8')
button4.place(x = 400, y = 50)

button5 = tk.Button(window, text = "疊放圖片", width = 10, height = 2, command = combined,bg='#c8c8c8')
button5.place(x = 400, y = 100)

button6 = tk.Button(window, text = "漣漪", width = 10, height = 2, command = rippleimg,bg='#c8c8c8')
button6.place(x = 400, y = 150)

button7 = tk.Button(window, text = "變亮", width = 10, height = 2, command = light,bg='#c8c8c8')
button7.place(x = 400, y = 200)

button8 = tk.Button(window, text = "使用說明", width = 10, height = 2, command = usetalk,bg='#c8c8c8')
button8.place(x = 200, y = 200)

window.mainloop()

RGBA
RGBA (1242, 1764)
